In [18]:
import pickle
import plotly.plotly as py
import plotly
import plotly.graph_objs as go

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [ ]:
from Bio import PDB
import numpy as np
import os
import cv2
#from pdb_utils import *

# Family Histogram

In [3]:
with open("pickle files/pdb_groups.pkl", "r") as f:
    groups = pickle.load(f)

In [4]:
histogram = dict(map(lambda x: (x[0],len(x[1])),groups.items()))

In [5]:
x,y = histogram.keys(), histogram.values()

In [6]:
init_notebook_mode(connected=True)

In [7]:
plotly.offline.plot({
"data": [
    go.Bar(x=x,y=y)
]
})

'file:///Users/hacertilbec/Documents/GitHub/Protein-Vectorization/Protein Distance Matrix/temp-plot.html'

In [8]:
print "total number of folds: ", len(histogram.keys())
print "total number of pdb files: ", reduce(lambda x,y: x+y, histogram.values())

total number of folds:  4344
total number of pdb files:  16022


# Pdb Vector Length Histogram

In [3]:
s_dict = {}
dir_path="PDBs"
files = os.listdir(dir_path)
pdb_files = [(f, os.path.join(dir_path, f)) for f in files if f.endswith(".pdb")]

for pdb, pdb_path in pdb_files:
    parser = PDB.PDBParser()
    structure = parser.get_structure(pdb, pdb_path)  
    s_size = 0
    coords_list = []
    model=structure[0]
    for chain in model.get_list():
        for residue in chain.get_list():
            try:
                coords = residue['CA'].coord
                s_size+=1
            except:
                continue
                
    s_dict[pdb] = s_size

/usr/local/lib/python2.7/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning:

Could not assign element 'X' for Atom (name=XE1) with given element 'X'

/usr/local/lib/python2.7/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning:

Could not assign element 'X' for Atom (name=XE2) with given element 'X'



In [4]:
len(s_dict.keys())

16020

In [5]:
hist_dict = {}
for key,value in s_dict.items():
    hist_dict.setdefault(value,0)
    hist_dict[value]+=1

In [6]:
len(hist_dict.keys())

584

In [7]:
x,y = hist_dict.keys(), hist_dict.values()

In [8]:
init_notebook_mode(connected=True)
layout = go.Layout(
    title='PDB Size Histogram',
)

plotly.offline.plot({
"data": [
    go.Bar(x=x,y=y)
]
})

'file:///Users/hacertilbec/Documents/GitHub/Protein-Vectorization/Protein Distance Matrix/temp-plot.html'

## Fold Histogram

In [9]:
folds = {}
for pclass, pdb_list in groups.items():
    fold = ".".join(pclass.split(".")[:2])
    for pdb in pdb_list:
        folds.setdefault(fold,[])
        folds[fold].append(pdb)

In [34]:

def pdb_size(pdb):
    try:
        dir_path="PDBs"
        pdb_path = os.path.join(dir_path, pdb+".pdb")
        parser = PDB.PDBParser()
        structure = parser.get_structure(pdb, pdb_path)  
        s_size = 0
        model=structure[0]
        for chain in model.get_list():
            for residue in chain.get_list():
                try:
                    coords = residue['CA'].coord
                    s_size+=1
                except:
                    continue
        return s_size
    except:
        return np.nan

In [ ]:
fold_hist = map(lambda x:(x[0],map(lambda y:pdb_size(y), x[1])), folds.items())

In [ ]:
sorted_mean = sorted(map(lambda x: x[0],mean[x[1]],fold_hist), key=lambda tup: tup[1])
sorted_mean

In [23]:
x = fold_hist.keys()
y = map(lambda x: np.mean(x) ,fold_hist.values())

/usr/local/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python2.7/site-packages/numpy/core/_methods.py:80: RuntimeWarning:

invalid value encountered in double_scalars



In [15]:
std_dev = map(lambda x: np.nanstd(x) ,fold_hist.values())

/usr/local/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:1434: RuntimeWarning:

Degrees of freedom <= 0 for slice.



In [16]:
mins = map(lambda x: min(x) ,fold_hist.values())
maxs = map(lambda x: max(x) ,fold_hist.values()) 

In [17]:
data = [
    go.Scatter(
        x=x,
        y=y,
        error_y=dict(
            type='data',
            array=std_dev
        )
    )
]

plotly.offline.plot(data, filename='temp.html')

'file:///Users/hacertilbec/Documents/GitHub/Protein-Vectorization/Protein Distance Matrix/temp.html'

In [29]:
trace1 = go.Scatter(
    x=x,
    y=mins,
    name='min ',
    mode = 'markers',
    marker = dict(size = 5)
)
trace2 = go.Scatter(
    x=x,
    y=maxs,
    name='max',
    yaxis='y2',
    mode = 'markers',
    marker = dict(size = 5)
)
trace3 = go.Scatter(
    x=x,
    y=y,
    name="mean",
    yaxis="y3",
    mode = 'markers',
    marker = dict(size = 5),
    error_y=dict(
        type='data',
        thickness=1.5,
        array=std_dev
    )
)
data = [trace1,trace2, trace3]

layout = go.Layout(
    title='Fold based pdb size analysis',
    yaxis2=dict(overlaying='y'),
    yaxis3=dict(overlaying='y')
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename='temp2.html')

TypeError: __init__() got an unexpected keyword argument 'shared_yaxes'

In [26]:
with open("pickle files/fold_groups.pkl", "r") as f:
    folds = pickle.load(f)

In [27]:
folds

{'a.265': ['d2f6sb2', 'd3se5a_'],
 'a.179': ['d1no1a1'],
 'f.62': ['d2kdcc_', 'd4uxza_'],
 'a.209': ['d2woea_', 'd3g9da_', 'd1t5ja1'],
 'f.60': ['d3q8ea1', 'd3teya1'],
 'f.61': ['d3mkua_', 'd4lz6a_'],
 'a.200': ['d1nxhb_'],
 'a.202': ['d3kphb_'],
 'a.203': ['d1yfsa1'],
 'a.204': ['d1yvwa1',
  'd1yxbb2',
  'd2a7wb_',
  'd3c90a_',
  'd1ogkd_',
  'd1w2ya_',
  'd2q9ld_',
  'd2yb0d_',
  'd4dk4b_',
  'd4qgpa1',
  'd2p06b1',
  'd1vmga_',
  'd2gtac1',
  'd2oigc1'],
 'a.205': ['d1us7b_'],
 'a.206': ['d1pp1x_'],
 'a.207': ['d1v9dd_', 'd1y64b_'],
 'd.65': ['d1tzpb_', 'd1r44a_', 'd2vo9a1'],
 'e.64': ['d2avuf_'],
 'b.104': ['d1hhna1', 'd1jhna3'],
 'a.216': ['d1r0di_', 'd1sj8a2', 'd1u89a1'],
 'a.49': ['d1f6va1'],
 'a.48': ['d3bi1a1',
  'd3kasa3',
  'd2okua1',
  'd1eo0a_',
  'd1wjta1',
  'd3buxb2',
  'd3vgoa1',
  'd3vrna1',
  'd3hphh1'],
 'd.368': ['d2h4ob_'],
 'd.369': ['d2icga1', 'd2paga1', 'd2prvb2'],
 'd.366': ['d2i9xa1', 'd2ia9c2'],
 'd.367': ['d2v5ga_', 'd3t7yb_', 'd5culb_'],
 'a.47': ['d4rd9b_